In [1]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-06-14 12:30:26.099266: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 12:30:26.142875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 12:30:26.822365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [4]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [5]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition import dataset
from swallowing_recognition import dence_net

In [14]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_neck_movement_folder = directory_path / 'washino' / 'neck_movement'
train_shoulder_movement_folder = directory_path / 'washino' / 'shoulder_movement'
train_arm_movement_folder = directory_path / 'washino' / 'arm_movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   
train_mumble_folder = directory_path / 'washino' / 'mumble'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    
test_movement_folder = directory_path / 'shibata' / 'movement'    
test_hitting_folder = directory_path / 'shibata' / 'hitting'    

# 引数は(データ数，画像サイズ縦，画像サイズ横，色RGB，クラス数)
# train_data = dataset.DataSet(500, 224, 224, 3, 5)
# test_data = dataset.DataSet(70, 224, 224, 3, 5)

train_data = dataset.DataSet(2150, 224, 224, 3, 2)
test_data = dataset.DataSet(62, 224, 224, 3, 2)

# train_data.folder_to_dataset(train_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# train_data.folder_to_dataset(train_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# train_data.folder_to_dataset(train_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# train_data.folder_to_dataset(train_movement_folder, np.array([0, 1, 0, 0, 0]), 3)
# train_data.folder_to_dataset(train_hitting_folder, np.array([1, 0, 0, 0, 0]), 4)

train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0)

In [15]:
train_data.folder_to_dataset(train_voice_folder, np.array(1), 700)
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 800)
train_data.folder_to_dataset(train_cough_folder, np.array(1), 900)
train_data.folder_to_dataset(train_neck_movement_folder, np.array(1), 1000)
train_data.folder_to_dataset(train_shoulder_movement_folder, np.array(1), 1100)
train_data.folder_to_dataset(train_arm_movement_folder, np.array(1), 1200)
# train_data.folder_to_dataset(train_mumble_folder, np.array(1), 1300)

In [16]:
names = ['haruki', 'ezumi', 'hikaru', 'ibuki', 'kanata', 'kazuki', 'kishimoto', 'kosasa', 'mocchi', 'nakaryo', 'ruku', 'shibata', 'soichiro', 'tsuji', 'genki', 'kim', 'kyogoku']
print(len(names))

17


In [9]:
exit_name = 'haruki'

if exit_name in names:
    index_to_pop = names.index(exit_name)
    names.pop(index_to_pop)  # インデックスを使用して要素を削除
    print(f"{exit_name} has been removed.")
    print(names)
else:
    print(f"{exit_name} not found in the list.")

haruki has been removed.
['ezumi', 'hikaru', 'ibuki', 'kanata', 'kazuki', 'kishimoto', 'kosasa', 'mocchi', 'nakaryo', 'ruku', 'shibata', 'soichiro', 'tsuji']


In [17]:
start_num = 1300
for name in names:
    prediction_swallowing_folder = directory_path / name / 'swallowing'
    prediction_voice_folder = directory_path / name / 'voice'
    prediction_cough_folder = directory_path / name / 'cough'
    prediction_hitting_folder = directory_path / name / 'hitting'    
    prediction_movement_folder = directory_path / name / 'movement' 
    
    train_data.folder_to_dataset(prediction_swallowing_folder, np.array(0), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_voice_folder, np.array(1), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_cough_folder, np.array(1), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_hitting_folder, np.array(1), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_movement_folder, np.array(1), start_num)
    start_num += 10
    print(start_num)

1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150


In [18]:
test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data.folder_to_dataset(test_voice_folder, np.array(1), 28)
test_data.folder_to_dataset(test_movement_folder, np.array(1), 42)
test_data.folder_to_dataset(test_hitting_folder, np.array(1), 52)

In [19]:
print(type(train_data.data))
print((train_data.data).shape)
print((train_data.labels).shape)

<class 'numpy.ndarray'>
(2150, 224, 224, 3)
(2150,)


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data.data, train_data.labels, test_size=0.2, random_state=42)

# 結果の形状を確認
print("Train data shape:", X_train.shape)  # トレーニングデータの形状
print("Test data shape:", X_test.shape)    # テストデータの形状
print("Train labels shape:", y_train.shape) # トレーニングデータのラベルの形状
print("Test labels shape:", y_test.shape)   # テストデータのラベルの形状

count_0 = np.count_nonzero(y_train == 0)
count_1 = np.count_nonzero(y_train == 1)

print("train number of 0s:", count_0)
print("train number of 1s:", count_1)

count_0 = np.count_nonzero(y_test == 0)
count_1 = np.count_nonzero(y_test == 1)

print("test number of 0s:", count_0)
print("test number of 1s:", count_1)

Train data shape: (1720, 224, 224, 3)
Test data shape: (430, 224, 224, 3)
Train labels shape: (1720,)
Test labels shape: (430,)
train number of 0s: 696
train number of 1s: 1024
test number of 0s: 174
test number of 1s: 256


In [21]:
from sklearn.model_selection import KFold
import numpy as np

# 5分割交差検証の設定
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# データを保持するためのリスト
train_data_splits = []
test_data_splits = []

# データのインデックスを分割
for train_index, test_index in kf.split(train_data.data):
    X_train, X_test = train_data.data[train_index], train_data.data[test_index]
    y_train, y_test = train_data.labels[train_index], train_data.labels[test_index]

    # 各フォールドのデータをリストに保存
    train_data_splits.append((X_train, y_train))
    test_data_splits.append((X_test, y_test))

# 結果の使用例
for i, (train, test) in enumerate(zip(train_data_splits, test_data_splits)):
    X_train, y_train = train
    X_test, y_test = test
    print(f"Fold {i+1}")
    print("Train data shape:", X_train.shape)
    print("Test data shape:", X_test.shape)
    print("Train labels shape:", y_train.shape)
    print("Test labels shape:", y_test.shape)
    print("Train number of 0s:", np.count_nonzero(y_train == 0))
    print("Train number of 1s:", np.count_nonzero(y_train == 1))
    print("Test number of 0s:", np.count_nonzero(y_test == 0))
    print("Test number of 1s:", np.count_nonzero(y_test == 1))



Fold 1
Train data shape: (1720, 224, 224, 3)
Test data shape: (430, 224, 224, 3)
Train labels shape: (1720,)
Test labels shape: (430,)
Train number of 0s: 696
Train number of 1s: 1024
Test number of 0s: 174
Test number of 1s: 256
Fold 2
Train data shape: (1720, 224, 224, 3)
Test data shape: (430, 224, 224, 3)
Train labels shape: (1720,)
Test labels shape: (430,)
Train number of 0s: 687
Train number of 1s: 1033
Test number of 0s: 183
Test number of 1s: 247
Fold 3
Train data shape: (1720, 224, 224, 3)
Test data shape: (430, 224, 224, 3)
Train labels shape: (1720,)
Test labels shape: (430,)
Train number of 0s: 673
Train number of 1s: 1047
Test number of 0s: 197
Test number of 1s: 233
Fold 4
Train data shape: (1720, 224, 224, 3)
Test data shape: (430, 224, 224, 3)
Train labels shape: (1720,)
Test labels shape: (430,)
Train number of 0s: 715
Train number of 1s: 1005
Test number of 0s: 155
Test number of 1s: 275
Fold 5
Train data shape: (1720, 224, 224, 3)
Test data shape: (430, 224, 224, 3)

In [23]:
# model = dence_net.DanceNet(5)

model = dence_net.DanceNet(2)

early_stopping = EarlyStopping(
    monitor='val_loss',  # 監視する値
    patience=10,         # 性能が改善されないエポック数
    verbose=1,           # 進行状況のメッセージ表示
    mode='min',          # 「減少」を監視
    restore_best_weights=True  # 最良のモデルの重みを復元
)

model_checkpoint = ModelCheckpoint(
    filepath='20240614_binary_model_best_all.keras',  # 保存するファイルのパス
    monitor='accuracy',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='max'                 # 監視
)

# model_checkpoint = ModelCheckpoint(
#     filepath='except_' + exit_name + '_20240507_binary_model_best.keras',  # 保存するファイルのパス
#     monitor='accuracy',        # 監視する値
#     save_best_only=True,       # 最良のモデルのみを保存
#     verbose=1,                 # 進行状況のメッセージ表示
#     mode='max'                 # 監視
# )

# model_checkpoint = ModelCheckpoint(
#     filepath='20240213_binary_model_best.keras',  # 保存するファイルのパス
#     monitor = "accuracy",
#     save_best_only=True,       # 最良のモデルのみを保存
#     verbose=1,                 # 進行状況のメッセージ表示
#     mode='max'                 # 「減少」を監視
# )

# 引数は(トレーニングデータ，ラベル，epochs，batch_size)
# model.training(train_data.data, train_data.labels, 40, 32, model_checkpoint = model_checkpoint)
# model.evaluate(test_data.data, test_data.labels)
# model.save('except_' + exit_name + '20240507_binary_ep40_bs32_v1.keras')

2024-06-14 15:29:58.158127: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22038 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [28]:
use_data = 4

X_train, y_train = train_data_splits[use_data] 
X_test, y_test = test_data_splits[use_data]

model.training(X_train, y_train, 40, 32, model_checkpoint = model_checkpoint)
model.evaluate(X_test, y_test)
model.save('20240614_binary_ep40_bs32_4.keras')

Epoch 1/40
49/49 [==============================] - ETA: 0s - loss: 0.0257 - accuracy: 0.9897
Epoch 1: accuracy did not improve from 1.00000
49/49 [==============================] - 6s 127ms/step - loss: 0.0257 - accuracy: 0.9897 - val_loss: 0.4121 - val_accuracy: 0.9244
Epoch 2/40
49/49 [==============================] - ETA: 0s - loss: 0.0126 - accuracy: 0.9948
Epoch 2: accuracy did not improve from 1.00000
49/49 [==============================] - 6s 127ms/step - loss: 0.0126 - accuracy: 0.9948 - val_loss: 0.3561 - val_accuracy: 0.9360
Epoch 3/40
49/49 [==============================] - ETA: 0s - loss: 0.0212 - accuracy: 0.9916
Epoch 3: accuracy did not improve from 1.00000
49/49 [==============================] - 6s 125ms/step - loss: 0.0212 - accuracy: 0.9916 - val_loss: 0.4095 - val_accuracy: 0.9302
Epoch 4/40
49/49 [==============================] - ETA: 0s - loss: 0.0075 - accuracy: 0.9974
Epoch 4: accuracy did not improve from 1.00000
49/49 [==============================] - 6s

In [29]:
model.training(train_data.data, train_data.labels, 40, 32, model_checkpoint = model_checkpoint)
model.save('202400614_binary_ep40_bs32_all.keras')

Epoch 1/40
61/61 [==============================] - ETA: 0s - loss: 0.0182 - accuracy: 0.9964
Epoch 1: accuracy did not improve from 1.00000
61/61 [==============================] - 12s 199ms/step - loss: 0.0182 - accuracy: 0.9964 - val_loss: 0.6899 - val_accuracy: 0.9070
Epoch 2/40
61/61 [==============================] - ETA: 0s - loss: 0.1209 - accuracy: 0.9612
Epoch 2: accuracy did not improve from 1.00000
61/61 [==============================] - 8s 124ms/step - loss: 0.1209 - accuracy: 0.9612 - val_loss: 0.3809 - val_accuracy: 0.9256
Epoch 3/40
61/61 [==============================] - ETA: 0s - loss: 0.0389 - accuracy: 0.9866
Epoch 3: accuracy did not improve from 1.00000
61/61 [==============================] - 8s 125ms/step - loss: 0.0389 - accuracy: 0.9866 - val_loss: 0.3689 - val_accuracy: 0.9163
Epoch 4/40
61/61 [==============================] - ETA: 0s - loss: 0.0162 - accuracy: 0.9933
Epoch 4: accuracy did not improve from 1.00000
61/61 [==============================] - 8

Epoch 31/40
61/61 [==============================] - ETA: 0s - loss: 6.6636e-05 - accuracy: 1.0000
Epoch 31: accuracy did not improve from 1.00000
61/61 [==============================] - 8s 126ms/step - loss: 6.6636e-05 - accuracy: 1.0000 - val_loss: 0.3426 - val_accuracy: 0.9395
Epoch 32/40
61/61 [==============================] - ETA: 0s - loss: 5.7764e-05 - accuracy: 1.0000
Epoch 32: accuracy did not improve from 1.00000
61/61 [==============================] - 8s 129ms/step - loss: 5.7764e-05 - accuracy: 1.0000 - val_loss: 0.3483 - val_accuracy: 0.9349
Epoch 33/40
61/61 [==============================] - ETA: 0s - loss: 5.3885e-05 - accuracy: 1.0000
Epoch 33: accuracy did not improve from 1.00000
61/61 [==============================] - 8s 129ms/step - loss: 5.3885e-05 - accuracy: 1.0000 - val_loss: 0.3495 - val_accuracy: 0.9349
Epoch 34/40
61/61 [==============================] - ETA: 0s - loss: 3.5256e-05 - accuracy: 1.0000
Epoch 34: accuracy did not improve from 1.00000
61/61 [

In [ ]:
# model.evaluate_print()
test_data2 = dataset.DataSet(42, 224, 224, 3, 2)
test_data2.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data2.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data2.folder_to_dataset(test_voice_folder, np.array(1), 28)

In [ ]:
model.evaluate(test_data2.data, test_data2.labels)